<h1>Creating Geography lookup tables</h1>

This file creates a postcode locations from the NSPL_May_2022_UK.<br>
This output can be be joined to tables read in using area/district/sector. <br>

In [ ]:
project_path = "/home/jupyter"
import sys
sys.path.append(project_path)

import numpy as np
import pandas as pd

from fintrans_toolbox.src import bq_utils as bq
from google.cloud import bigquery

In [ ]:
client = bigquery.Client()

In [ ]:
#Read in NSPL
sql_nspl = """SELECT DISTINCT pcds, doterm, rgn, ctry, ru11ind, imd
FROM ons-fintrans-data-prod.fintrans_reference.NSPL_MAY_2022_UK"""

df_nspl = bq.read_bq_table_sql(client, sql_nspl)
#Remove postcodes that terminated pre 2019
df_nspl = df_nspl[(df_nspl.doterm >= 201901) |  (df_nspl.doterm.isna())]

In [ ]:
#Caluclate area, district, sector
df_nspl[['postal_district','postal_sector']] = df_nspl["pcds"].str.split(" ", 1, expand=True)
df_nspl['postal_sector'] = df_nspl['postal_sector'].str.strip()
df_nspl['postal_sector'] = df_nspl['postal_sector'].str[:1]
df_nspl['postal_sector'] = df_nspl['postal_district'] + " " +  df_nspl['postal_sector']
df_nspl['postal_area'] = df_nspl['postal_district'].str.replace('\d+', '')
df_nspl['postal_area'] = df_nspl['postal_area'].str[:2]

#Reorder columns
cols = df_nspl.columns.tolist()
cols = cols[-3:] + cols[:-3]
df_nspl = df_nspl[cols] 

In [ ]:
df_nspl.to_csv("~/fintrans_toolbox/data/NSPL_location_lookup.csv")